In [3]:
import pandas as pd
from haversine import haversine
import rapidfuzz
import warnings
warnings.filterwarnings("ignore")
from tqdm.auto import tqdm
from itertools import combinations
import numpy as np
from utils.explode import explode_df
from utils.text_clean import clean_text
from Config import config

In [4]:
df = pd.read_csv('/workspace/clustering/data/input/Fuse_exploded_ZAF_cleaned.csv')

df["poi"] = df["placeId"]

In [5]:
df["brands"].fillna('UNKwn',inplace =True)
df["brands"] = df["brands"].map(str)

In [6]:
def isNameMatch(name1, name2):
    numerator = rapidfuzz.distance.Levenshtein.distance(name1.lower(), name2.lower()) * 100
    denominator = len(name1) if len(name1) > len(name2) else len(name2)
    
    #Foloowing statement checks for common words between 2 strings and mark containent flag as true or false
    containment = len(set(name1.lower().split()) & set(name2.lower().split())) > 0
    
    if ((numerator / denominator) <= 50) and (containment==False):
        return False
    else:
        return True

In [7]:
def isBrandMatch(brand1, brand2):
    #match = True
    if brand1 == "UNKwn" or brand2 == "UNKwn":
        return True
    elif brand1.lower() != brand2.lower():
        return False
    elif brand1.lower() == brand2.lower():
        return True

In [8]:
def isDistanceMatch(lat1, lon1, lat2, lon2):
    distance = haversine((lat1, lon1), (lat2, lon2), unit='m')
    if distance > 500:
        return False
    else:
        return True

In [9]:
matches = set()
for cluster_id, values in tqdm(df.groupby(["clusterId"])):
    names = values["placeId"].tolist()
    comb = set([tuple(sorted(x)) for x in set(combinations(names, 2))])
    matches.update(comb)

  0%|          | 0/673053 [00:00<?, ?it/s]

In [10]:
df_pairs = pd.DataFrame(matches)
df_pairs.columns = ['Id1', 'Id2']
df_pairs.drop_duplicates(inplace=True)

In [11]:

df_pairs = pd.merge(df_pairs, df,  how='left', left_on=[
                    'Id1'], right_on=['placeId'])
df_pairs = pd.merge(df_pairs, df,  how='left', left_on=[
                    'Id2'], right_on=['placeId'], suffixes=[1, 2])

In [11]:
df_pairs.columns

Index(['placeId1', 'placeId2', 'locality1', 'clusterId1', 'placeId1',
       'sourceNames1', 'category1', 'brands1', 'latitude1', 'longitude1',
       'houseNumber1', 'streets1', 'cities1', 'postalCode1', 'Id1', 'poi1',
       'locality2', 'clusterId2', 'placeId2', 'sourceNames2', 'category2',
       'brands2', 'latitude2', 'longitude2', 'houseNumber2', 'streets2',
       'cities2', 'postalCode2', 'Id2', 'poi2'],
      dtype='object')

In [12]:
df_pairs["isNameMatch"] = np.vectorize(isNameMatch)(df_pairs['sourceNames1'], df_pairs['sourceNames2'])
df_pairs["isBrandMatch"] = np.vectorize(isNameMatch)(df_pairs['brands1'], df_pairs['brands2'])
df_pairs["isDistanceMatch"] = np.vectorize(isDistanceMatch)(df_pairs['latitude1'], df_pairs['longitude1'],df_pairs['latitude2'], df_pairs['longitude2'])


In [32]:
df_pairs.columns

Index(['Id1', 'Id2', 'locality1', 'clusterId1', 'placeId1', 'sourceNames1',
       'category1', 'brands1', 'latitude1', 'longitude1', 'houseNumber1',
       'streets1', 'cities1', 'postalCode1', 'Id1', 'poi1', 'locality2',
       'clusterId2', 'placeId2', 'sourceNames2', 'category2', 'brands2',
       'latitude2', 'longitude2', 'houseNumber2', 'streets2', 'cities2',
       'postalCode2', 'Id2', 'poi2', 'isNameMatch', 'isBrandMatch',
       'isDistanceMatch'],
      dtype='object')

In [13]:
match_result = df_pairs.groupby(['isNameMatch', 'isBrandMatch', 'isDistanceMatch'], as_index=False)['placeId1'].count()
match_result

,isNameMatch,isBrandMatch,isDistanceMatch,placeId1
0,False,False,False,5
1,False,False,True,28
2,False,True,False,780
3,False,True,True,6769
4,True,False,False,689
5,True,False,True,401
6,True,True,False,424343
7,True,True,True,4933971


In [14]:
true_value = match_result[(match_result['isNameMatch']==True) & \
                          (match_result['isBrandMatch']==True) & \
                          (match_result['isDistanceMatch']==True)]['placeId1'].values[0]
true_value

4933971

In [15]:
match_result['percentage'] = (match_result['placeId1']/true_value)*100
match_result

,isNameMatch,isBrandMatch,isDistanceMatch,placeId1,percentage
0,False,False,False,5,0.000101
1,False,False,True,28,0.000567
2,False,True,False,780,0.015809
3,False,True,True,6769,0.137192
4,True,False,False,689,0.013964
5,True,False,True,401,0.008127
6,True,True,False,424343,8.600436
7,True,True,True,4933971,100.000000


# Model output calculation

In [16]:
df_clusters = pd.read_csv("/workspace/results/NZL_final_clustered.csv")

In [18]:
df_raw = pd.read_csv("/workspace/data/raw/Fuse_NZL.csv")

In [19]:
cols = ['locality','placeId', 'sourceNames',
       'rawCategories', 'insertedCategories', 'preemptiveCategories',
       'category', 'subCategory', 'officialName', 'brands', 'phoneNumbers',
       'internet', 'email', 'latitude', 'longitude', 'houseNumber', 'streets',
       'cities', 'postalCode']

In [20]:
df = pd.merge(df_clusters, df_raw[cols],  how='left', left_on=[
                    'placeId'], right_on=['placeId'], suffixes=[1, 2])

In [21]:
df = explode_df(df)

length of overall dataframe 495781
length of overall dataframe after exploding sourceNames is 514055
length of overall dataframe after dropping null 514055
length of after exploding all columns 995663
length after dropping where lat and long both are zero 985233
length after dropping null sourcenames 985233
length after dropping duplicates 973953
length of final explode 973953


In [22]:
df["brands"].fillna('UNKwn',inplace =True)
df["brands"] = df["brands"].map(str)

In [23]:
df["latitude"] = df["latitude"].map(float)
df["longitude"] = df["longitude"].map(float)


In [24]:
df["sourceNames"] = df["sourceNames"].map(clean_text)
df['sourceNames'] = df['sourceNames'].replace('', np.nan)

In [25]:

print(f'----in drop columns--')
for col in config.drop_cols:
    print(f'--------current col : {col}')
    df = df.dropna(subset=[col])

df['houseNumber'] = df['houseNumber'].replace('', np.nan)
df['streets'] = df['streets'].replace('', np.nan)
df['category'] = df['category'].replace('', np.nan)
df['postalCode'] = df['postalCode'].replace('', np.nan)
df['cities'] = df['cities'].replace('', np.nan)

df = imputing_values(df, dict_update_null=config.dict_update_null)

df["streets"] = df["streets"].map(clean_text)
df["cities"] = df["cities"].map(clean_text)
df["category"] = df["category"].map(clean_text)
df["houseNumber"] = df["houseNumber"].map(clean_text)
df["postalCode"] = df["postalCode"].map(clean_text)
df = imputing_values(df, dict_update_null=config.dict_update_null)

df.reset_index(drop=True, inplace=True)
df["Id"] = df.index

for col in df.columns:
    if col not in config.columns_keep:
        df.drop(col,axis=1,inplace=True)
    

----in drop columns--
--------current col : sourceNames
--------current col : latitude
--------current col : longitude
----in imputing values--
->->current col : houseNumber, impute value :0
 null values length for the column houseNumber is 286606
length of index list : 286606
->->current col : streets, impute value : 
 null values length for the column streets is 121297
length of index list : 407903
->->current col : cities, impute value : 
 null values length for the column cities is 8334
length of index list : 416237
->->current col : postalCode, impute value :0
 null values length for the column postalCode is 22608
length of index list : 438845
----in imputing values--
->->current col : houseNumber, impute value :0
 null values length for the column houseNumber is 0
length of index list : 0
->->current col : streets, impute value : 
 null values length for the column streets is 0
length of index list : 0
->->current col : cities, impute value : 
 null values length for the column c

In [26]:
matches = set()
for cluster_id, values in tqdm(df.groupby(["clusterId"])):
    names = values["placeId"].tolist()
    comb = set([tuple(sorted(x)) for x in set(combinations(names, 2))])
    matches.update(comb)

  0%|          | 0/299899 [00:00<?, ?it/s]

In [27]:
df_pairs = pd.DataFrame(matches)
df_pairs.columns = ['Id1', 'Id2']
df_pairs.drop_duplicates(inplace=True)

In [28]:

df_pairs = pd.merge(df_pairs, df,  how='left', left_on=[
                    'Id1'], right_on=['placeId'])
df_pairs = pd.merge(df_pairs, df,  how='left', left_on=[
                    'Id2'], right_on=['placeId'], suffixes=[1, 2])

In [29]:
df_pairs["isNameMatch"] = np.vectorize(isNameMatch)(df_pairs['sourceNames1'], df_pairs['sourceNames2'])
df_pairs["isBrandMatch"] = np.vectorize(isNameMatch)(df_pairs['brands1'], df_pairs['brands2'])
df_pairs["isDistanceMatch"] = np.vectorize(isDistanceMatch)(df_pairs['latitude1'], df_pairs['longitude1'],df_pairs['latitude2'], df_pairs['longitude2'])


In [30]:
match_result = df_pairs.groupby(['isNameMatch', 'isBrandMatch', 'isDistanceMatch'], as_index=False)['placeId1'].count()
match_result

,isNameMatch,isBrandMatch,isDistanceMatch,placeId1
0,False,False,False,6
1,False,False,True,38
2,False,True,False,170
3,False,True,True,5495
4,True,False,False,30
5,True,False,True,769
6,True,True,False,64698
7,True,True,True,3135106


In [31]:
true_value = match_result[(match_result['isNameMatch']==True) & \
                          (match_result['isBrandMatch']==True) & \
                          (match_result['isDistanceMatch']==True)]['placeId1'].values[0]
true_value

3135106

In [ ]:
# False	True	False	18	0.029819	0.029819
# 1	False	True	True	200	0.331323	0.331323
# 2	True	True	False	3637	6.025114	6.025114
# 3	True	True	True	60364	100.000000	100.000000

In [32]:
match_result['percentage'] = (match_result['placeId1']/true_value)*100
match_result

,isNameMatch,isBrandMatch,isDistanceMatch,placeId1,percentage
0,False,False,False,6,0.000191
1,False,False,True,38,0.001212
2,False,True,False,170,0.005422
3,False,True,True,5495,0.175273
4,True,False,False,30,0.000957
5,True,False,True,769,0.024529
6,True,True,False,64698,2.063662
7,True,True,True,3135106,100.000000
